In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os

# Define CNN Architecture
class CNN(nn.Module):
    def __init__(self, activation='relu', init='xavier'):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)  # Assuming input images are 128x128 after resizing
        self.fc2 = nn.Linear(256, 2)  # 2 classes: Cat and Dog
        self.dropout = nn.Dropout(0.5)
        self.batch_norm = nn.BatchNorm2d(128)
        
        self.activation = self.get_activation(activation)
        self.init_weights(init)

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = self.pool(self.activation(self.conv3(x)))
        x = self.batch_norm(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x
    
    def get_activation(self, name):
        if name == 'relu':
            return F.relu
        elif name == 'tanh':
            return F.tanh
        elif name == 'leaky_relu':
            return F.leaky_relu
        else:
            raise ValueError("Unsupported activation function")
    
    def init_weights(self, init_type):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                if init_type == 'xavier':
                    nn.init.xavier_uniform_(m.weight)
                elif init_type == 'kaiming':
                    nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                elif init_type == 'random':
                    nn.init.uniform_(m.weight, -0.1, 0.1)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

# Prepare DataLoader for Cats vs. Dogs Dataset
def get_cats_dogs_dataloader(batch_size=64):
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    # Make sure the path is correct
    train_set = datasets.ImageFolder(root='/kaggle/input/cat-and-dog/training_set/training_set', transform=transform)
    test_set = datasets.ImageFolder(root='/kaggle/input/cat-and-dog/test_set/test_set', transform=transform)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

# Training Function
def train_model(model, train_loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
    return model

# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

# Configurations for Optimizer and Loss Function
optimizers = {'sgd': optim.SGD, 'adam': optim.Adam, 'rmsprop': optim.RMSprop}
criterion = nn.CrossEntropyLoss()

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare data loaders for Cats vs. Dogs
train_loader, test_loader = get_cats_dogs_dataloader()

# Experimenting with different optimizers, activations, and initializations
best_accuracy = 0
best_model = None

# Train the model with different configurations
for opt_name, opt_fn in optimizers.items():
    for activation in ['relu', 'tanh', 'leaky_relu']:
        for init in ['xavier', 'kaiming', 'random']:
            print(f"Training CNN with optimizer: {opt_name}, activation: {activation}, init: {init}")
            model = CNN(activation, init).to(device)
            optimizer = opt_fn(model.parameters(), lr=0.001)
            model = train_model(model, train_loader, optimizer, criterion, epochs=10)
            acc = evaluate_model(model, test_loader)
            if acc > best_accuracy:
                best_accuracy = acc
                best_model = model

# Save the best model
torch.save(best_model.state_dict(), "best_cnn_cats_dogs_model.pth")
print(f"Best CNN model saved with accuracy: {best_accuracy}")


Training CNN with optimizer: sgd, activation: relu, init: xavier
Epoch [1/10], Loss: 0.8642
Epoch [2/10], Loss: 0.6485
Epoch [3/10], Loss: 0.5654
Epoch [4/10], Loss: 0.5190
Epoch [5/10], Loss: 0.4829
Epoch [6/10], Loss: 0.4508
Epoch [7/10], Loss: 0.4231
Epoch [8/10], Loss: 0.3996
Epoch [9/10], Loss: 0.3802
Epoch [10/10], Loss: 0.3524
Test Accuracy: 72.17%
Training CNN with optimizer: sgd, activation: relu, init: kaiming
Epoch [1/10], Loss: 0.9204
Epoch [6/10], Loss: 0.4380
Epoch [7/10], Loss: 0.4149
Epoch [8/10], Loss: 0.3832
Epoch [9/10], Loss: 0.3581
Epoch [10/10], Loss: 0.3327
Test Accuracy: 70.64%
Training CNN with optimizer: sgd, activation: relu, init: random
Epoch [1/10], Loss: 2.8454
Epoch [2/10], Loss: 1.1920
Epoch [3/10], Loss: 0.7943
Epoch [4/10], Loss: 0.7154
Epoch [5/10], Loss: 0.6946
Epoch [6/10], Loss: 0.6777
Epoch [7/10], Loss: 0.6691
Epoch [8/10], Loss: 0.6753
Epoch [9/10], Loss: 0.6508
Epoch [10/10], Loss: 0.6595
Test Accuracy: 63.57%
Training CNN with optimizer: sgd,

# **Fine-Tune Resnet-18**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import os
from torchvision import models

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fine-Tune ResNet-18 for CIFAR-10 with Extra Layers
def fine_tune_resnet18():
    model = models.resnet18(pretrained=True)

    # Unfreeze the last 2 blocks for fine-tuning
    for param in model.parameters():
        param.requires_grad = False  # Freeze all layers

    for param in list(model.layer4.parameters()):  # Unfreeze the last block
        param.requires_grad = True

    for param in list(model.layer3.parameters()):  # Unfreeze the second last block
        param.requires_grad = True

    num_ftrs = model.fc.in_features

    # Modify the classifier with extra layers
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 1024),  
        nn.LeakyReLU(negative_slope=0.01),
        nn.BatchNorm1d(1024),
        nn.Linear(1024, 512),
        nn.LeakyReLU(negative_slope=0.01),
        nn.BatchNorm1d(512),
        nn.Dropout(0.3),
        nn.Linear(512, 2)  # Output layer for Cats vs. Dogs (2 classes)
    )
    
    return model.to(device)

# Prepare DataLoader for Cats vs. Dogs Dataset
def get_cats_dogs_dataloader(batch_size=64):
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    train_set = datasets.ImageFolder(root='/kaggle/input/cat-and-dog/training_set/training_set', transform=transform)
    test_set = datasets.ImageFolder(root='/kaggle/input/cat-and-dog/test_set/test_set', transform=transform)
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader

# Training Function
def train_model(model, train_loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
    return model

# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

# Load Data
train_loader, test_loader = get_cats_dogs_dataloader(batch_size=64)

# Configurations for Optimizer and Loss Function
optimizers = {'sgd': optim.SGD, 'adam': optim.Adam, 'rmsprop': optim.RMSprop}
criterion = nn.CrossEntropyLoss()

# Fine-tune ResNet-18
best_accuracy = 0
best_model = None

for opt_name, opt_fn in optimizers.items():
    print(f"Training ResNet-18 with optimizer: {opt_name}")
    model = fine_tune_resnet18()
    optimizer = opt_fn(model.parameters(), lr=0.001)
    model = train_model(model, train_loader, optimizer, criterion, epochs=10)
    acc = evaluate_model(model, test_loader)
    
    if acc > best_accuracy:
        best_accuracy = acc
        best_model = model

# Save the Best Model
torch.save(best_model.state_dict(), "best_resnet18_model.pth")
print("Best model saved with accuracy:", best_accuracy)


Training ResNet-18 with optimizer: sgd


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


Epoch [1/10], Loss: 0.4441
Epoch [2/10], Loss: 0.2698
Epoch [3/10], Loss: 0.2135
Epoch [4/10], Loss: 0.1816
Epoch [5/10], Loss: 0.1602
Epoch [6/10], Loss: 0.1597
Epoch [7/10], Loss: 0.1305
Epoch [8/10], Loss: 0.1268
Epoch [9/10], Loss: 0.1165
Epoch [10/10], Loss: 0.1044
Test Accuracy: 94.17%
Training ResNet-18 with optimizer: adam
Epoch [1/10], Loss: 0.1930
Epoch [2/10], Loss: 0.1214
Epoch [3/10], Loss: 0.0623
Epoch [4/10], Loss: 0.0409
Epoch [5/10], Loss: 0.0537
Epoch [6/10], Loss: 0.0603
Epoch [7/10], Loss: 0.0824
Epoch [8/10], Loss: 0.0287
Epoch [9/10], Loss: 0.0282
Epoch [10/10], Loss: 0.0345
Test Accuracy: 95.35%
Training ResNet-18 with optimizer: rmsprop
Epoch [1/10], Loss: 0.2603
Epoch [2/10], Loss: 0.1304
Epoch [3/10], Loss: 0.0876
Epoch [4/10], Loss: 0.0630
Epoch [5/10], Loss: 0.0823
Epoch [6/10], Loss: 0.0421
Epoch [7/10], Loss: 0.0320
Epoch [8/10], Loss: 0.0317
Epoch [9/10], Loss: 0.0637
Epoch [10/10], Loss: 0.0290
Test Accuracy: 93.13%
Best model saved with accuracy: 95.353